Notebook to create tables for paper:

- input: energy medians tables, execution time tables
- output: latex_nov/
    - energy_time_results.tex : table with energy and time
    - energy_time_results_aggregated.tex

In [114]:
import os
import itertools

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pingouin as pg
import scikit_posthocs as sp
import seaborn as sns
#from cliffs_delta import cliffs_delta
from scipy import stats


In [115]:
pwd

'D:\\GAISSA\\energy-repo\\repo_sept_last_update\\energy-ml-serving\\experiments'

In [116]:
SAVE_TABLES = True

In [117]:
# tables_cpuep_dir = 'D:/GAISSA/energy-repo/repo_sept_last/cpuep_processed/'
# tables_cudaep_dir = 'D:/GAISSA/energy-repo/repo_sept_last/cudaep_processed/'
#save_latex_dir = 'D:/GAISSA/energy-repo/repo_sept_last/overleaf_latex_tables/'

tables_cpuep_dir = 'D:/GAISSA/energy-repo/repo_sept_last_update/cpuep_nov_results/nov_cpuep_00/'
tables_cudaep_dir = 'D:/GAISSA/energy-repo/repo_sept_last_update/cudaep_nov_results/nov_cudaep_00/'
save_latex_dir = 'D:/GAISSA/energy-repo/repo_sept_last_update/latex_nov/'


variables =['global_energy',
                "avg_cpu_usage_config",'avg_used_memory_pct_config',
                'avg_utilization_gpu_config','avg_utilization_memory_config','avg_used_memory_pct_mib',
                'avg_Load', 'duration'
                ]
dependent_variable = 'avg_utilization_gpu_config'

In [118]:
def save_latex_table(df,save_dir):
    df = df.copy()
    df.columns = [col.replace('_', ' ') for col in df.columns]
    
    latex_table = df.to_latex(index=False)

    # Define the filename
    filename = save_dir

    if SAVE_TABLES:
        # Open the file in write mode
        with open(filename, 'w') as file:
            file.write(latex_table)
        
    # Print LaTeX table
    print(latex_table)

In [119]:
def remove_condition_rows(condition_to_remove,df):
    # example: condition = merged_df['label']  == 'idle_idle'
    df1 = df.copy()
    return df1[~condition_to_remove]

In [120]:
# change tables dir:
cpu_ep_data_energy = pd.read_csv(tables_cpuep_dir + "tables_nov/median_energy_values.csv", index_col=None, header=0)
cuda_ep_data_energy = pd.read_csv(tables_cudaep_dir + "tables_nov/median_energy_values.csv", index_col=None, header=0)

cpu_ep_data_time = pd.read_csv(tables_cpuep_dir + "tables_nov/median_time_inference.csv", index_col=None, header=0)
cuda_ep_data_time = pd.read_csv(tables_cudaep_dir + "tables_nov/median_time_inference.csv", index_col=None, header=0)

In [121]:
#cpu_ep_data_energy

## Energy

In [122]:
cpu_ep_data_energy['config'] = "cpuep_"+cpu_ep_data_energy['label']
cuda_ep_data_energy['config'] = "cudaep_"+cuda_ep_data_energy['label']

cuda_ep_data_energy

,runtime,model,label,experiment,CPU_energy,GPU_energy,global_energy,config
0,idle,idle,idle_idle,5.5,5331.372978,0.000000,5331.372978,cudaep_idle_idle
1,onnx,bloomz-1b1,onnx_bloomz-1b1,5.5,51748.184044,62642.135541,114327.170499,cudaep_onnx_bloomz-1b1
2,onnx,bloomz-560m,onnx_bloomz-560m,5.5,10006.586121,18373.828223,28371.988127,cudaep_onnx_bloomz-560m
3,onnx,codegemma-2b,onnx_codegemma-2b,5.5,35610.280609,77668.948789,113279.229398,cudaep_onnx_codegemma-2b
4,onnx,codeparrot-small,onnx_codeparrot-small,5.5,9081.845177,13986.934956,23077.985648,cudaep_onnx_codeparrot-small
5,onnx,phi2,onnx_phi2,5.5,40885.409996,87022.120454,127932.914739,cudaep_onnx_phi2
6,onnx,pythia-410m,onnx_pythia-410m,5.5,21988.519135,33065.796229,55228.025927,cudaep_onnx_pythia-410m
7,onnx,pythia1-4b,onnx_pythia1-4b,5.5,26553.710274,59124.014519,85651.403900,cudaep_onnx_pythia1-4b
8,onnx,stablecode-3b,onnx_stablecode-3b,5.5,32293.401276,54655.184992,86823.784125,cudaep_onnx_stablecode-3b
9,onnx,stablecode-3b-completion,onnx_stablecode-3b-completion,5.5,47259.743469,106721.017104,154000.752026,cudaep_onnx_stablecode-3b-completion


In [123]:
print(len(cpu_ep_data_energy))
condition = cpu_ep_data_energy['label'].str.contains('torchscript')
cpu_ep_data_energy = remove_condition_rows( condition, cpu_ep_data_energy)
print(len(cpu_ep_data_energy))

49
37


In [124]:
print(len(cuda_ep_data_energy))
condition = cuda_ep_data_energy['label'].str.contains('torchscript')
cuda_ep_data_energy = remove_condition_rows( condition, cuda_ep_data_energy)
print(len(cuda_ep_data_energy))

37
25


In [126]:
cpu_ep_data_energy['GPU_energy'] = 0
cpu_ep_data_energy['global_energy'] = cpu_ep_data_energy['CPU_energy']
cpu_ep_data_energy

,runtime,model,label,experiment,CPU_energy,GPU_energy,global_energy,config
0,idle,idle,idle_idle,5.5,5450.420303,0,5450.420303,cpuep_idle_idle
1,onnx,bloomz-1b1,onnx_bloomz-1b1,5.5,85002.455544,0,85002.455544,cpuep_onnx_bloomz-1b1
2,onnx,bloomz-560m,onnx_bloomz-560m,5.5,38111.320023,0,38111.320023,cpuep_onnx_bloomz-560m
3,onnx,codegemma-2b,onnx_codegemma-2b,5.5,303140.927048,0,303140.927048,cpuep_onnx_codegemma-2b
4,onnx,codeparrot-small,onnx_codeparrot-small,5.5,27370.594078,0,27370.594078,cpuep_onnx_codeparrot-small
5,onnx,phi2,onnx_phi2,5.5,475452.949699,0,475452.949699,cpuep_onnx_phi2
6,onnx,pythia-410m,onnx_pythia-410m,5.5,75764.569603,0,75764.569603,cpuep_onnx_pythia-410m
7,onnx,pythia1-4b,onnx_pythia1-4b,5.5,212320.958244,0,212320.958244,cpuep_onnx_pythia1-4b
8,onnx,stablecode-3b,onnx_stablecode-3b,5.5,218677.233612,0,218677.233612,cpuep_onnx_stablecode-3b
9,onnx,stablecode-3b-completion,onnx_stablecode-3b-completion,5.5,575477.807861,0,575477.807861,cpuep_onnx_stablecode-3b-completion


In [127]:
condition = cpu_ep_data_energy['config'].str.endswith('idle')
cpu_ep_data_energy = remove_condition_rows( condition, cpu_ep_data_energy)
print(len(cpu_ep_data_energy))

condition = cuda_ep_data_energy['config'].str.endswith('idle')
cuda_ep_data_energy = remove_condition_rows( condition, cuda_ep_data_energy)
print(len(cuda_ep_data_energy))

36
24


In [130]:
dependent_variable = ['model', 'config']
new_columns = ['model', 'config', 'global_energy']
cpu_ep_data_energy = cpu_ep_data_energy.sort_values(by=dependent_variable, ascending=True)[new_columns]
cpu_ep_data_energy

,model,config,global_energy
1,bloomz-1b1,cpuep_onnx_bloomz-1b1,85002.455544
13,bloomz-1b1,cpuep_ov_bloomz-1b1,73707.834908
25,bloomz-1b1,cpuep_torch_bloomz-1b1,60622.102364
2,bloomz-560m,cpuep_onnx_bloomz-560m,38111.320023
14,bloomz-560m,cpuep_ov_bloomz-560m,49575.380554
26,bloomz-560m,cpuep_torch_bloomz-560m,52217.206894
3,codegemma-2b,cpuep_onnx_codegemma-2b,303140.927048
15,codegemma-2b,cpuep_ov_codegemma-2b,409354.084854
27,codegemma-2b,cpuep_torch_codegemma-2b,334939.674240
4,codeparrot-small,cpuep_onnx_codeparrot-small,27370.594078


In [128]:
dependent_variable = ['model', 'config']
new_columns = ['model', 'config', 'CPU_energy', 'GPU_energy', 'global_energy']
cuda_ep_data_energy = cuda_ep_data_energy.sort_values(by=dependent_variable, ascending=True)[new_columns]
cuda_ep_data_energy

,model,config,CPU_energy,GPU_energy,global_energy
1,bloomz-1b1,cudaep_onnx_bloomz-1b1,51748.184044,62642.135541,114327.170499
13,bloomz-1b1,cudaep_torch_bloomz-1b1,2628.826508,9751.112565,12388.138642
2,bloomz-560m,cudaep_onnx_bloomz-560m,10006.586121,18373.828223,28371.988127
14,bloomz-560m,cudaep_torch_bloomz-560m,3254.764481,9976.271805,13233.709419
3,codegemma-2b,cudaep_onnx_codegemma-2b,35610.280609,77668.948789,113279.229398
15,codegemma-2b,cudaep_torch_codegemma-2b,10527.942001,42494.678662,53024.509396
4,codeparrot-small,cudaep_onnx_codeparrot-small,9081.845177,13986.934956,23077.985648
16,codeparrot-small,cudaep_torch_codeparrot-small,4424.862007,8426.396317,12825.179010
5,phi2,cudaep_onnx_phi2,40885.409996,87022.120454,127932.914739
17,phi2,cudaep_torch_phi2,12209.897324,51308.245523,63554.592497


### Together CPU and CUDA

In [84]:
energy_df = pd.concat([cpu_ep_data_energy,cuda_ep_data_energy], axis=0, ignore_index=True)
energy_df

,runtime,model,label,experiment,CPU_energy,GPU_energy,global_energy,config
0,idle,idle,idle_idle,5.5,5450.420303,0.000000,5450.420303,cpuep_idle_idle
1,onnx,bloomz-1b1,onnx_bloomz-1b1,5.5,85002.455544,0.000000,85002.455544,cpuep_onnx_bloomz-1b1
2,onnx,bloomz-560m,onnx_bloomz-560m,5.5,38111.320023,0.000000,38111.320023,cpuep_onnx_bloomz-560m
3,onnx,codegemma-2b,onnx_codegemma-2b,5.5,303140.927048,0.000000,303140.927048,cpuep_onnx_codegemma-2b
4,onnx,codeparrot-small,onnx_codeparrot-small,5.5,27370.594078,0.000000,27370.594078,cpuep_onnx_codeparrot-small
...,...,...,...,...,...,...,...,...
81,torchscript,stablecode-3b,torchscript_stablecode-3b,5.5,2344.199448,7663.673766,10002.734694,cudaep_torchscript_stablecode-3b
82,torchscript,stablecode-3b-completion,torchscript_stablecode-3b-completion,5.5,2380.344330,7504.766608,9874.418905,cudaep_torchscript_stablecode-3b-completion
83,torchscript,starcoderbase-1b,torchscript_starcoderbase-1b,5.5,1845.044197,6375.537556,8230.160399,cudaep_torchscript_starcoderbase-1b
84,torchscript,tiny-starcoder,torchscript_tiny-starcoder,5.5,1692.254761,5520.391939,7214.945183,cudaep_torchscript_tiny-starcoder


In [85]:
condition = energy_df['config'].str.endswith('idle')
energy_df = remove_condition_rows( condition, energy_df)
energy_df


,runtime,model,label,experiment,CPU_energy,GPU_energy,global_energy,config
1,onnx,bloomz-1b1,onnx_bloomz-1b1,5.5,85002.455544,0.000000,85002.455544,cpuep_onnx_bloomz-1b1
2,onnx,bloomz-560m,onnx_bloomz-560m,5.5,38111.320023,0.000000,38111.320023,cpuep_onnx_bloomz-560m
3,onnx,codegemma-2b,onnx_codegemma-2b,5.5,303140.927048,0.000000,303140.927048,cpuep_onnx_codegemma-2b
4,onnx,codeparrot-small,onnx_codeparrot-small,5.5,27370.594078,0.000000,27370.594078,cpuep_onnx_codeparrot-small
5,onnx,phi2,onnx_phi2,5.5,475452.949699,0.000000,475452.949699,cpuep_onnx_phi2
...,...,...,...,...,...,...,...,...
81,torchscript,stablecode-3b,torchscript_stablecode-3b,5.5,2344.199448,7663.673766,10002.734694,cudaep_torchscript_stablecode-3b
82,torchscript,stablecode-3b-completion,torchscript_stablecode-3b-completion,5.5,2380.344330,7504.766608,9874.418905,cudaep_torchscript_stablecode-3b-completion
83,torchscript,starcoderbase-1b,torchscript_starcoderbase-1b,5.5,1845.044197,6375.537556,8230.160399,cudaep_torchscript_starcoderbase-1b
84,torchscript,tiny-starcoder,torchscript_tiny-starcoder,5.5,1692.254761,5520.391939,7214.945183,cudaep_torchscript_tiny-starcoder


In [86]:
dependent_variable = ['model', 'config']
new_columns = ['model', 'config', 'CPU_energy', 'GPU_energy', 'global_energy']
energy_df_to_save = energy_df.sort_values(by=dependent_variable, ascending=True)[new_columns]
energy_df_to_save

,model,config,CPU_energy,GPU_energy,global_energy
1,bloomz-1b1,cpuep_onnx_bloomz-1b1,85002.455544,0.000000,85002.455544
13,bloomz-1b1,cpuep_ov_bloomz-1b1,73707.834908,0.000000,73707.834908
25,bloomz-1b1,cpuep_torch_bloomz-1b1,60622.102364,0.000000,60622.102364
37,bloomz-1b1,cpuep_torchscript_bloomz-1b1,13192.124016,0.000000,13192.124016
50,bloomz-1b1,cudaep_onnx_bloomz-1b1,51748.184044,62642.135541,114327.170499
...,...,...,...,...,...
36,tinyllama,cpuep_torch_tinyllama,233383.063507,0.000000,233383.063507
48,tinyllama,cpuep_torchscript_tinyllama,11293.706779,0.000000,11293.706779
61,tinyllama,cudaep_onnx_tinyllama,20832.141495,46703.842778,67800.167767
73,tinyllama,cudaep_torch_tinyllama,8958.797241,27899.111313,36827.716835


## Execution time

In [132]:
cpu_ep_data_time['config'] = "cpuep_"+cpu_ep_data_time['label']
cuda_ep_data_time['config'] = "cudaep_"+cuda_ep_data_time['label']

cuda_ep_data_time

,runtime,model,label,duration,config
0,onnx,bloomz-1b1,onnx_bloomz-1b1,1029.257360,cudaep_onnx_bloomz-1b1
1,onnx,bloomz-560m,onnx_bloomz-560m,225.435832,cudaep_onnx_bloomz-560m
2,onnx,codegemma-2b,onnx_codegemma-2b,678.768990,cudaep_onnx_codegemma-2b
3,onnx,codeparrot-small,onnx_codeparrot-small,191.048335,cudaep_onnx_codeparrot-small
4,onnx,phi2,onnx_phi2,793.478886,cudaep_onnx_phi2
5,onnx,pythia-410m,onnx_pythia-410m,447.527716,cudaep_onnx_pythia-410m
6,onnx,pythia1-4b,onnx_pythia1-4b,535.354030,cudaep_onnx_pythia1-4b
7,onnx,stablecode-3b,onnx_stablecode-3b,617.516664,cudaep_onnx_stablecode-3b
8,onnx,stablecode-3b-completion,onnx_stablecode-3b-completion,964.568877,cudaep_onnx_stablecode-3b-completion
9,onnx,starcoderbase-1b,onnx_starcoderbase-1b,556.800123,cudaep_onnx_starcoderbase-1b


In [88]:
time_df = pd.concat([cpu_ep_data_time,cuda_ep_data_time], axis=0, ignore_index=True)
time_df

,runtime,model,label,duration,config
0,onnx,bloomz-1b1,onnx_bloomz-1b1,1399.077517,cpuep_onnx_bloomz-1b1
1,onnx,bloomz-560m,onnx_bloomz-560m,615.157070,cpuep_onnx_bloomz-560m
2,onnx,codegemma-2b,onnx_codegemma-2b,3721.251811,cpuep_onnx_codegemma-2b
3,onnx,codeparrot-small,onnx_codeparrot-small,400.491458,cpuep_onnx_codeparrot-small
4,onnx,phi2,onnx_phi2,4454.560860,cpuep_onnx_phi2
...,...,...,...,...,...
79,torchscript,stablecode-3b,torchscript_stablecode-3b,98.837918,cudaep_torchscript_stablecode-3b
80,torchscript,stablecode-3b-completion,torchscript_stablecode-3b-completion,98.986819,cudaep_torchscript_stablecode-3b-completion
81,torchscript,starcoderbase-1b,torchscript_starcoderbase-1b,94.641855,cudaep_torchscript_starcoderbase-1b
82,torchscript,tiny-starcoder,torchscript_tiny-starcoder,94.731452,cudaep_torchscript_tiny-starcoder


In [89]:
condition = time_df['config'].str.endswith('idle')
time_df = remove_condition_rows( condition, time_df)

In [90]:
dependent_variable = ['model', 'config']
new_columns = ['model', 'config', 'duration']
time_df_to_save = time_df.sort_values(by=dependent_variable, ascending=True)[new_columns]
time_df_to_save

,model,config,duration
0,bloomz-1b1,cpuep_onnx_bloomz-1b1,1399.077517
12,bloomz-1b1,cpuep_ov_bloomz-1b1,575.626701
24,bloomz-1b1,cpuep_torch_bloomz-1b1,731.558769
36,bloomz-1b1,cpuep_torchscript_bloomz-1b1,126.630455
48,bloomz-1b1,cudaep_onnx_bloomz-1b1,1029.257360
...,...,...,...
35,tinyllama,cpuep_torch_tinyllama,2963.893108
47,tinyllama,cpuep_torchscript_tinyllama,94.712956
59,tinyllama,cudaep_onnx_tinyllama,401.162165
71,tinyllama,cudaep_torch_tinyllama,188.817668


In [91]:
energy_df.join(time_df.set_index('config'), on='config', how='left', lsuffix='_left', rsuffix='_right')

,runtime_left,model_left,label_left,experiment,CPU_energy,GPU_energy,global_energy,config,runtime_right,model_right,label_right,duration
1,onnx,bloomz-1b1,onnx_bloomz-1b1,5.5,85002.455544,0.000000,85002.455544,cpuep_onnx_bloomz-1b1,onnx,bloomz-1b1,onnx_bloomz-1b1,1399.077517
2,onnx,bloomz-560m,onnx_bloomz-560m,5.5,38111.320023,0.000000,38111.320023,cpuep_onnx_bloomz-560m,onnx,bloomz-560m,onnx_bloomz-560m,615.157070
3,onnx,codegemma-2b,onnx_codegemma-2b,5.5,303140.927048,0.000000,303140.927048,cpuep_onnx_codegemma-2b,onnx,codegemma-2b,onnx_codegemma-2b,3721.251811
4,onnx,codeparrot-small,onnx_codeparrot-small,5.5,27370.594078,0.000000,27370.594078,cpuep_onnx_codeparrot-small,onnx,codeparrot-small,onnx_codeparrot-small,400.491458
5,onnx,phi2,onnx_phi2,5.5,475452.949699,0.000000,475452.949699,cpuep_onnx_phi2,onnx,phi2,onnx_phi2,4454.560860
...,...,...,...,...,...,...,...,...,...,...,...,...
81,torchscript,stablecode-3b,torchscript_stablecode-3b,5.5,2344.199448,7663.673766,10002.734694,cudaep_torchscript_stablecode-3b,torchscript,stablecode-3b,torchscript_stablecode-3b,98.837918
82,torchscript,stablecode-3b-completion,torchscript_stablecode-3b-completion,5.5,2380.344330,7504.766608,9874.418905,cudaep_torchscript_stablecode-3b-completion,torchscript,stablecode-3b-completion,torchscript_stablecode-3b-completion,98.986819
83,torchscript,starcoderbase-1b,torchscript_starcoderbase-1b,5.5,1845.044197,6375.537556,8230.160399,cudaep_torchscript_starcoderbase-1b,torchscript,starcoderbase-1b,torchscript_starcoderbase-1b,94.641855
84,torchscript,tiny-starcoder,torchscript_tiny-starcoder,5.5,1692.254761,5520.391939,7214.945183,cudaep_torchscript_tiny-starcoder,torchscript,tiny-starcoder,torchscript_tiny-starcoder,94.731452


In [92]:
df_to_save = energy_df.join(time_df.set_index('config'), on='config', how='left', lsuffix='_left', rsuffix='_right')
df_to_save.columns

Index(['runtime_left', 'model_left', 'label_left', 'experiment', 'CPU_energy',
       'GPU_energy', 'global_energy', 'config', 'runtime_right', 'model_right',
       'label_right', 'duration'],
      dtype='object')

In [93]:
columns_to_save  = ['model_left', 'config', 'CPU_energy','GPU_energy', 'global_energy', 'duration']

df_to_save = df_to_save[columns_to_save]
df_to_save.sort_values(by=['model_left','config'], ascending=True)

,model_left,config,CPU_energy,GPU_energy,global_energy,duration
1,bloomz-1b1,cpuep_onnx_bloomz-1b1,85002.455544,0.000000,85002.455544,1399.077517
13,bloomz-1b1,cpuep_ov_bloomz-1b1,73707.834908,0.000000,73707.834908,575.626701
25,bloomz-1b1,cpuep_torch_bloomz-1b1,60622.102364,0.000000,60622.102364,731.558769
37,bloomz-1b1,cpuep_torchscript_bloomz-1b1,13192.124016,0.000000,13192.124016,126.630455
50,bloomz-1b1,cudaep_onnx_bloomz-1b1,51748.184044,62642.135541,114327.170499,1029.257360
...,...,...,...,...,...,...
36,tinyllama,cpuep_torch_tinyllama,233383.063507,0.000000,233383.063507,2963.893108
48,tinyllama,cpuep_torchscript_tinyllama,11293.706779,0.000000,11293.706779,94.712956
61,tinyllama,cudaep_onnx_tinyllama,20832.141495,46703.842778,67800.167767,401.162165
73,tinyllama,cudaep_torch_tinyllama,8958.797241,27899.111313,36827.716835,188.817668


In [94]:
df_to_save['config'].unique()

array(['cpuep_onnx_bloomz-1b1', 'cpuep_onnx_bloomz-560m',
       'cpuep_onnx_codegemma-2b', 'cpuep_onnx_codeparrot-small',
       'cpuep_onnx_phi2', 'cpuep_onnx_pythia-410m',
       'cpuep_onnx_pythia1-4b', 'cpuep_onnx_stablecode-3b',
       'cpuep_onnx_stablecode-3b-completion',
       'cpuep_onnx_starcoderbase-1b', 'cpuep_onnx_tiny-starcoder',
       'cpuep_onnx_tinyllama', 'cpuep_ov_bloomz-1b1',
       'cpuep_ov_bloomz-560m', 'cpuep_ov_codegemma-2b',
       'cpuep_ov_codeparrot-small', 'cpuep_ov_phi2',
       'cpuep_ov_pythia-410m', 'cpuep_ov_pythia1-4b',
       'cpuep_ov_stablecode-3b', 'cpuep_ov_stablecode-3b-completion',
       'cpuep_ov_starcoderbase-1b', 'cpuep_ov_tiny-starcoder',
       'cpuep_ov_tinyllama', 'cpuep_torch_bloomz-1b1',
       'cpuep_torch_bloomz-560m', 'cpuep_torch_codegemma-2b',
       'cpuep_torch_codeparrot-small', 'cpuep_torch_phi2',
       'cpuep_torch_pythia-410m', 'cpuep_torch_pythia1-4b',
       'cpuep_torch_stablecode-3b',
       'cpuep_torch_stablecode

In [95]:
def remove_after_second_underscore(value):
    # Split the string by '_', take only the first three parts, and join them back with '_'
    return '_'.join(value.split('_')[:2])

In [96]:
df_to_save['config'] = df_to_save['config'].apply(remove_after_second_underscore)
df_to_save

,model_left,config,CPU_energy,GPU_energy,global_energy,duration
1,bloomz-1b1,cpuep_onnx,85002.455544,0.000000,85002.455544,1399.077517
2,bloomz-560m,cpuep_onnx,38111.320023,0.000000,38111.320023,615.157070
3,codegemma-2b,cpuep_onnx,303140.927048,0.000000,303140.927048,3721.251811
4,codeparrot-small,cpuep_onnx,27370.594078,0.000000,27370.594078,400.491458
5,phi2,cpuep_onnx,475452.949699,0.000000,475452.949699,4454.560860
...,...,...,...,...,...,...
81,stablecode-3b,cudaep_torchscript,2344.199448,7663.673766,10002.734694,98.837918
82,stablecode-3b-completion,cudaep_torchscript,2380.344330,7504.766608,9874.418905,98.986819
83,starcoderbase-1b,cudaep_torchscript,1845.044197,6375.537556,8230.160399,94.641855
84,tiny-starcoder,cudaep_torchscript,1692.254761,5520.391939,7214.945183,94.731452


In [97]:
df_to_save['config'].unique()

array(['cpuep_onnx', 'cpuep_ov', 'cpuep_torch', 'cpuep_torchscript',
       'cudaep_onnx', 'cudaep_torch', 'cudaep_torchscript'], dtype=object)

In [98]:
# Mapping dictionary
#mapping = {'cpuep_onnx': 'C1', 'cpuep_ov': 'C2', 'cpuep_torch': 'C3','cpuep_torchscript': 'C4','cudaep_onnx': 'C5', 'cudaep_torch': 'C6', 'cudaep_torchscript': 'C7',}
mapping = {'cpuep_onnx': '<CPU, ONNX>', 'cpuep_ov': '<CPU, OV>', 'cpuep_torch': '<CPU, TORCH>','cpuep_torchscript': '<CPU, JIT>',
          'cudaep_onnx': '<CUDA, ONNX>', 'cudaep_torch': '<CUDA, TORCH>', 'cudaep_torchscript': '<CUDA, JIT>',}

# Apply mapping to the 'category' column
df_to_save['config'] = df_to_save['config'].map(mapping)
df_to_save = df_to_save.rename(columns={'model_left': 'model'})
df_to_save.sort_values(by=['model','config'], ascending=True)

,model,config,CPU_energy,GPU_energy,global_energy,duration
37,bloomz-1b1,"<CPU, JIT>",13192.124016,0.000000,13192.124016,126.630455
1,bloomz-1b1,"<CPU, ONNX>",85002.455544,0.000000,85002.455544,1399.077517
13,bloomz-1b1,"<CPU, OV>",73707.834908,0.000000,73707.834908,575.626701
25,bloomz-1b1,"<CPU, TORCH>",60622.102364,0.000000,60622.102364,731.558769
74,bloomz-1b1,"<CUDA, JIT>",1948.270256,6212.964742,8163.522848,95.870545
...,...,...,...,...,...,...
24,tinyllama,"<CPU, OV>",257276.523949,0.000000,257276.523949,1500.745790
36,tinyllama,"<CPU, TORCH>",233383.063507,0.000000,233383.063507,2963.893108
85,tinyllama,"<CUDA, JIT>",2543.364494,7057.923739,9535.773841,103.467378
61,tinyllama,"<CUDA, ONNX>",20832.141495,46703.842778,67800.167767,401.162165


In [99]:
save_latex_table(df_to_save.sort_values(by=['model','config'], ascending=True) , save_latex_dir + 'energy_time_results.tex')

\begin{tabular}{llrrrr}
\toprule
model & config & CPU energy & GPU energy & global energy & duration \\
\midrule
bloomz-1b1 & <CPU, JIT> & 13192.124016 & 0.000000 & 13192.124016 & 126.630455 \\
bloomz-1b1 & <CPU, ONNX> & 85002.455544 & 0.000000 & 85002.455544 & 1399.077517 \\
bloomz-1b1 & <CPU, OV> & 73707.834908 & 0.000000 & 73707.834908 & 575.626701 \\
bloomz-1b1 & <CPU, TORCH> & 60622.102364 & 0.000000 & 60622.102364 & 731.558769 \\
bloomz-1b1 & <CUDA, JIT> & 1948.270256 & 6212.964742 & 8163.522848 & 95.870545 \\
bloomz-1b1 & <CUDA, ONNX> & 51748.184044 & 62642.135541 & 114327.170499 & 1029.257360 \\
bloomz-1b1 & <CUDA, TORCH> & 2628.826508 & 9751.112565 & 12388.138642 & 105.560827 \\
bloomz-560m & <CPU, JIT> & 9037.917389 & 0.000000 & 9037.917389 & 94.503039 \\
bloomz-560m & <CPU, ONNX> & 38111.320023 & 0.000000 & 38111.320023 & 615.157070 \\
bloomz-560m & <CPU, OV> & 49575.380554 & 0.000000 & 49575.380554 & 455.812262 \\
bloomz-560m & <CPU, TORCH> & 52217.206894 & 0.000000 & 52217

In [100]:
save_latex_dir

'D:/GAISSA/energy-repo/repo_sept_last_update/latex_nov/'

In [103]:
SAVE_TABLES

True

In [104]:
if SAVE_TABLES:
    df_to_save.sort_values(by=['model','config'], ascending=True).to_csv(save_latex_dir + "energy_time_results.csv", index=False)

In [105]:
df_to_save['model'].unique()

array(['bloomz-1b1', 'bloomz-560m', 'codegemma-2b', 'codeparrot-small',
       'phi2', 'pythia-410m', 'pythia1-4b', 'stablecode-3b',
       'stablecode-3b-completion', 'starcoderbase-1b', 'tiny-starcoder',
       'tinyllama'], dtype=object)

In [106]:
# Updated model and size data
model_data = {
    'Model': [
        'codeparrot-small', 'tiny-starcoder', 'pythia-410m', 'bloomz-560m',
        'starcoderbase-1b', 'bloomz-1b1', 'tinyllama', 'pythia1-4b',
        'codegemma-2b', 'phi2', 'stablecode-3b', 'stablecode-3b-completion'
    ],
    'Size': [110, 164, 410, 560, 1000, 1100, 1100, 1400, 2000, 2700, 3000, 3000]  # Model sizes in millions
}

# Create a new DataFrame for models
model_df = pd.DataFrame(model_data)

# Define size ranges based on provided groups
def assign_size_range(size):
    if size < 1000:
        return 1
    elif 1000 <= size < 2000:
        return 2
    else:
        return 3

model_df['size_group'] = model_df['Size'].apply(assign_size_range)
model_df

,Model,Size,size_group
0,codeparrot-small,110,1
1,tiny-starcoder,164,1
2,pythia-410m,410,1
3,bloomz-560m,560,1
4,starcoderbase-1b,1000,2
5,bloomz-1b1,1100,2
6,tinyllama,1100,2
7,pythia1-4b,1400,2
8,codegemma-2b,2000,3
9,phi2,2700,3


In [107]:

# Map the Size Range from model_df to df_to_save
df_to_save_02 = df_to_save.merge(model_df[['Model', 'size_group']], left_on='model', right_on='Model', how='left')

# Drop the redundant 'Model' column after merging
df_to_save_02 = df_to_save_02.drop(columns=['Model'])
df_to_save_02

,model,config,CPU_energy,GPU_energy,global_energy,duration,size_group
0,bloomz-1b1,"<CPU, ONNX>",85002.455544,0.000000,85002.455544,1399.077517,2
1,bloomz-560m,"<CPU, ONNX>",38111.320023,0.000000,38111.320023,615.157070,1
2,codegemma-2b,"<CPU, ONNX>",303140.927048,0.000000,303140.927048,3721.251811,3
3,codeparrot-small,"<CPU, ONNX>",27370.594078,0.000000,27370.594078,400.491458,1
4,phi2,"<CPU, ONNX>",475452.949699,0.000000,475452.949699,4454.560860,3
...,...,...,...,...,...,...,...
79,stablecode-3b,"<CUDA, JIT>",2344.199448,7663.673766,10002.734694,98.837918,3
80,stablecode-3b-completion,"<CUDA, JIT>",2380.344330,7504.766608,9874.418905,98.986819,3
81,starcoderbase-1b,"<CUDA, JIT>",1845.044197,6375.537556,8230.160399,94.641855,2
82,tiny-starcoder,"<CUDA, JIT>",1692.254761,5520.391939,7214.945183,94.731452,1


In [108]:
for c in df_to_save_02.columns:
    print(type(df_to_save_02.iloc[0][c]))

<class 'str'>
<class 'str'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.int64'>


In [110]:
if SAVE_TABLES:
    df_to_save_02.sort_values(by=['size_group','model','config'], ascending=True).to_csv(save_latex_dir + "energy_time_results_range.csv", index=False) 


df_to_save_02.sort_values(by=['size_group','model','config'], ascending=True)

,model,config,CPU_energy,GPU_energy,global_energy,duration,size_group
37,bloomz-560m,"<CPU, JIT>",9037.917389,0.000000,9037.917389,94.503039,1
1,bloomz-560m,"<CPU, ONNX>",38111.320023,0.000000,38111.320023,615.157070,1
13,bloomz-560m,"<CPU, OV>",49575.380554,0.000000,49575.380554,455.812262,1
25,bloomz-560m,"<CPU, TORCH>",52217.206894,0.000000,52217.206894,660.673025,1
73,bloomz-560m,"<CUDA, JIT>",1858.257210,6286.553030,8143.872922,95.592857,1
...,...,...,...,...,...,...,...
20,stablecode-3b-completion,"<CPU, OV>",585458.906937,0.000000,585458.906937,3239.888109,3
32,stablecode-3b-completion,"<CPU, TORCH>",631788.437874,0.000000,631788.437874,7379.742725,3
80,stablecode-3b-completion,"<CUDA, JIT>",2380.344330,7504.766608,9874.418905,98.986819,3
56,stablecode-3b-completion,"<CUDA, ONNX>",47259.743469,106721.017104,154000.752026,964.568877,3


In [111]:
# Selecting only numeric columns for grouping
numeric_columns = ['CPU_energy', 'GPU_energy', 'global_energy', 'duration',]
grouped_by_size_range = df_to_save_02.groupby(['size_group','config'])[numeric_columns].mean().reset_index()
grouped_by_size_range

,size_group,config,CPU_energy,GPU_energy,global_energy,duration
0,1,"<CPU, JIT>",5120.847706,0.000000,5120.847706,94.444604
1,1,"<CPU, ONNX>",45069.354385,0.000000,45069.354385,655.092825
2,1,"<CPU, OV>",58374.875793,0.000000,58374.875793,561.605482
3,1,"<CPU, TORCH>",64993.753622,0.000000,64993.753622,773.272561
4,1,"<CUDA, JIT>",1876.043694,5938.361817,7807.141250,95.673640
5,1,"<CUDA, ONNX>",13404.990673,20692.811506,34138.272887,277.122695
6,1,"<CUDA, TORCH>",5193.326574,11250.868661,16432.524734,124.034311
7,2,"<CPU, JIT>",12491.193190,0.000000,12491.193190,103.005501
8,2,"<CPU, ONNX>",178641.042351,0.000000,178641.042351,2100.828593
9,2,"<CPU, OV>",223821.618273,0.000000,223821.618273,1355.182816


In [113]:
if SAVE_TABLES:
    grouped_by_size_range.to_csv(save_latex_dir + 'energy_time_results_aggregated.csv', index=False) 
    save_latex_table(grouped_by_size_range , save_latex_dir + 'energy_time_results_aggregated.tex')

\begin{tabular}{rlrrrr}
\toprule
size group & config & CPU energy & GPU energy & global energy & duration \\
\midrule
1 & <CPU, JIT> & 5120.847706 & 0.000000 & 5120.847706 & 94.444604 \\
1 & <CPU, ONNX> & 45069.354385 & 0.000000 & 45069.354385 & 655.092825 \\
1 & <CPU, OV> & 58374.875793 & 0.000000 & 58374.875793 & 561.605482 \\
1 & <CPU, TORCH> & 64993.753622 & 0.000000 & 64993.753622 & 773.272561 \\
1 & <CUDA, JIT> & 1876.043694 & 5938.361817 & 7807.141250 & 95.673640 \\
1 & <CUDA, ONNX> & 13404.990673 & 20692.811506 & 34138.272887 & 277.122695 \\
1 & <CUDA, TORCH> & 5193.326574 & 11250.868661 & 16432.524734 & 124.034311 \\
2 & <CPU, JIT> & 12491.193190 & 0.000000 & 12491.193190 & 103.005501 \\
2 & <CPU, ONNX> & 178641.042351 & 0.000000 & 178641.042351 & 2100.828593 \\
2 & <CPU, OV> & 223821.618273 & 0.000000 & 223821.618273 & 1355.182816 \\
2 & <CPU, TORCH> & 216854.715006 & 0.000000 & 216854.715006 & 2583.765204 \\
2 & <CUDA, JIT> & 2155.506113 & 6622.720055 & 8760.154779 & 97.8646